<a href="https://colab.research.google.com/github/Danc2050/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <strong> <h2> Colab GPU and CPU Notebook</strong></h2>  
#{ output-height: 8, form-width: "100%", display-mode: "form" }
#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>🔌 Make sure the GPU is enabled under <em>Runtime->Change runtime type</em>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in a browser to prevent disconnection.</h4>
#@markdown <h4>🕷️ The <em>debug</em> field shows CUDALucas progress and GPU specs.</h4>
#@markdown <h4>📜 See the <a href="https://github.com/Danc2050/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">source code</a> for more information.</h4>
#@markdown <h4>🤷‍♂️ Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here.</a></h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
prime_password = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
GPU_type_of_work = '100 - First time LL tests' #@param ['100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests']
CPU_type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '154 - 100 million digit PRP tests']
debug = "False" #@param ["False", "True (exits after completion)"]

# Load & cd into gdrive for persistent data
from google.colab import drive
import os
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'

def run():
  # Run CUDALucas and Mprime
  !cd cudalucas; nohup python2 primenet.py -d -T $GPU_type_of_work -u $prime_ID -p $prime_password & # Start PrimeNet to get/submit work.
  !sleep 1.5
  !cd cudalucas && chmod 777 CUDALucas; nohup ./CUDALucas & # Outputs to nohup.out in cudalucas directory
  !cd mprime_gpu && chmod 777 mprime && ./mprime -d

def install():
  # Download/Install/Configure CUDALucas
  !wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
  !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/cudalucas.sh -q # download CUDALucas script
  !sed -i '/^if ! COMPUTE=/,/^fi/!b; /^fi/a echo "$COMPUTE"' cudalucas.sh # Output CUDA compute capability of GPU
  !sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_35,code=sm_35 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' cudalucas.sh
  !sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas.sh # Do not check for an Nvidia GPU
  !sed -i '/^nohup / s/^/# /' cudalucas.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/# /' cudalucas.sh # Do not create a cronjob
  !bash -- cudalucas.sh "" $prime_ID $GPU_type_of_work # Do not pass any additional parameters here

  # Download/Install/Configure Prime95
  !rm mprime.sh # remove any old download of the Prime95 script
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/mprime.sh -q # Download Prime95 script
  !sed -i '/^nohup / s/^/# /' mprime.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime.sh # Do not create a cronjob
  !sed -i 's/"mprime"/"mprime_gpu"/' mprime.sh # Name the folder specific to the runtime type
  !bash -- mprime.sh $prime_ID $computer_name $CPU_type_of_work # Run script
  run()

def debug_exit():
  !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this notebook
  !tail -n 100 cudalucas/nohup.out # view CUDALucas progress

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
prime_password = 'vikings' if prime_password.lower() == 'default' else prime_password
CPU_type_of_work = CPU_type_of_work.split("-")[0].rstrip()
GPU_type_of_work = GPU_type_of_work.split("-")[0].rstrip()
debug = False if debug == 'False' else True

# Add quotes to string args so script can parse spaces/special characters
prime_ID = f'"'+ prime_ID + f'"'
computer_name = f'"'+ computer_name + f'"'
prime_password = f'"'+ prime_password + f'"'

# GPU Check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the "Runtime" -> "Change runtime type" -> "GPU" -> "SAVE" to enable a GPU accelerator, ')
    print('and then re-execute this cell.')

if debug:
  debug_exit()

elif os.path.exists('mprime_gpu') and os.path.exists('cudalucas') :
  run()

else:
  install()

print("Gracefully exiting...")